<a href="https://colab.research.google.com/github/agnibrainhack/AmexModel/blob/master/AmEx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from google.colab import files
uploaded = files.upload()

Saving final_train.zip to final_train.zip


In [8]:
from zipfile import ZipFile
 
# specifying the zip file name
file_name = "final_train.zip"
 
# opening the zip file in READ mode
with ZipFile(file_name, 'r') as zip:
    # printing all the contents of the zip file
    zip.printdir()
 
    # extracting all the files
    print('Extracting all the files now...')
    zip.extractall()
    print('Done!')
    zip.close()

File Name                                             Modified             Size
final_train.csv                                2018-11-17 14:30:40     49636843
Extracting all the files now...
Done!


In [0]:

import numpy as np
import pandas as pd
from keras.models import load_model
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Dropout, Activation
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger, Callback  #early stopping ta aktu pore implement korchi
from sklearn.metrics import roc_auc_score
from keras.callbacks import Callback
from sklearn.model_selection import train_test_split
import keras
import pandas as pd

In [10]:
!ls

final_train.csv  final_train.zip  sample_data


In [0]:
train = pd.read_csv('final_train.csv')
X = train.drop(columns=['is_click'])
Y = train[['is_click']]
# X , Y = loadData()
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=0)

In [0]:
class roc_callback(Callback):
    def __init__(self,training_data,validation_data):
        self.x = training_data[0]
        self.y = training_data[1]
        self.x_val = validation_data[0]
        self.y_val = validation_data[1]


    def on_train_begin(self, logs={}):
        return

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict(self.x)
        roc = roc_auc_score(self.y, y_pred)
        y_pred_val = self.model.predict(self.x_val)
        roc_val = roc_auc_score(self.y_val, y_pred_val)
        print('\rroc-auc: %s - roc-auc_val: %s' % (str(round(roc,4)),str(round(roc_val,4))),end=100*' '+'\n')
        return

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

In [0]:
def base_model():
        model = Sequential()
        model.add(Dense(250, input_dim=X_train.shape[1],
                        kernel_initializer=keras.initializers.he_normal(seed=None)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.3))
        
        model.add(Dense(125, kernel_initializer=keras.initializers.he_normal(seed=None)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.2))
        
        model.add(Dense(85, kernel_initializer=keras.initializers.he_normal(seed=None)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.2))
        
        model.add(Dense(35, kernel_initializer=keras.initializers.he_normal(seed=None)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.1))
        
        model.add(Dense(10, kernel_initializer=keras.initializers.he_normal(seed=None)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
#         model.add(Dropout(0.08))
        model.add(Dense(1, activation='sigmoid'))

        # Compile model
        opt = keras.optimizers.adam(lr=0.01, decay=1e-7)
        model.compile(optimizer=opt, metrics = ['accuracy'], loss='binary_crossentropy')

        return model

In [14]:
model = base_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 250)               17500     
_________________________________________________________________
activation_1 (Activation)    (None, 250)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 250)               1000      
_________________________________________________________________
dropout_1 (Dropout)          (None, 250)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 125)               31375     
_________________________________________________________________
activation_2 (Activation)    (None, 125)               0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 125)               500       
__________

In [0]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=64,shuffle=True, 
          callbacks=[roc_callback(training_data=(X_train, y_train),validation_data=(X_test, y_test))])

Train on 267383 samples, validate on 29710 samples
Epoch 1/50
267383/267383 [==============================] - 104s 388us/step - loss: 0.2721 - acc: 0.9230 - val_loss: 0.2663 - val_acc: 0.9251
roc-auc: 0.5251 - roc-auc_val: 0.5357                                                                                                    
Epoch 2/50
267383/267383 [==============================] - 100s 373us/step - loss: 0.2691 - acc: 0.9240 - val_loss: 0.2651 - val_acc: 0.9251
roc-auc: 0.5503 - roc-auc_val: 0.5543                                                                                                    
Epoch 3/50
267383/267383 [==============================] - 100s 376us/step - loss: 0.2686 - acc: 0.9240 - val_loss: 0.2649 - val_acc: 0.9251
roc-auc: 0.5581 - roc-auc_val: 0.5589                                                                                                    
Epoch 4/50
267383/267383 [==============================] - 100s 373us/step - loss: 0.2680 - acc: 0.9240 - va